In [11]:
import geopandas as gpd
import xarray as xr
import numpy as np
import rioxarray as rxr
import os
from math import e
import matplotlib.pyplot as plt
import pandas as pd

In [59]:
#IDir = r'/Volumes/PhD/imagery/masters/ROC/M1_ROC.shp'
IDir = r'/Volumes/PhD/imagery/masters/output/ROC/design_storm/M1_design_storm.shp'
ODir = r'/Volumes/PhD/imagery/masters/output/ROC/design_storm/'
dNBR_df = gpd.read_file(IDir)
dNBR_df
for row in dNBR_df:
    print(row['BASIN_ID'])

TypeError: string indices must be integers

In [53]:
def M1(dnbr, Value):
    
    # Coefficients for Southern California
    b = -3.63
    b_1 = 0.41
    b_2 = 0.67
    b_3 = 0.7
    
    x_1 = dnbr['L_X1']
    x_3 = dnbr['L_X3']
    x_2 = dnbr[f'{Value}']
    #R = dnbr['RI']
    lnx =  b + (b_1 * x_1 * RI) + (b_2 * x_2 * RI) + (b_3 * x_3 * RI)
    prob = (e ** lnx) / (1.0 + e ** lnx)
    return prob
    

In [ ]:
mean = []
median = []
peak_f = []
R = [3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0]
for RI in R:
    M1P_Mean = M1(dNBR_df, 'Mean_dNBR')
    mean.append(M1P_Mean)
    M1P_Median = M1(dNBR_df, 'Med_dNBR')
    M1P_PF = M1(dNBR_df, 'PF_dNBR')




0       0.064164
1       0.102490
2       0.117073
3       0.115671
4       0.201483
          ...   
1731    0.060798
1732    0.067893
1733    0.081784
1734    0.225089
1735    0.148464
Length: 1736, dtype: float64

In [ ]:
M1P_df = {
    'M1P_Mean': M1P_Mean,
    'M1P_Med': M1P_Median,
    'M1p_PF': M1P_PF
}
M1P_df = pd.DataFrame(M1P_df)

merged_gdf = gpd.GeoDataFrame(
    pd.merge(dNBR_df, M1P_df, left_index=True, right_index=True, how="left")
)

24


In [21]:
merged_gdf

,BASIN_ID,Shape_Leng,Shape_Area,Segment_ID,Acc_Cl,Perim_Buff,InsidePeri,OutsidePer,Fire_ID,Fire_Segme,...,CombHaz,CombHazCl,CombHazCl_,Mean_dNBR,Med_dNBR,PF_dNBR,geometry,M1P_Mean,M1P_Med,M1p_PF
0,123,16060.0,6.322700e+06,123,1.0,10,0,0,thm2017,thm2017_123,...,3,1,Low,0.135826,0.068758,0.042093,"POLYGON ((278199.84 3833189.559, 278169.84 383...",0.386199,0.286454,0.251368
1,155,3140.0,2.716000e+05,155,1.0,10,101,12,thm2017,thm2017_155,...,2,1,Low,0.260572,0.151689,0.063711,"POLYGON ((278489.84 3833309.559, 278529.84 383...",0.775065,0.624249,0.479554
2,167,1800.0,7.260000e+04,167,1.0,10,100,11,thm2017,thm2017_167,...,2,1,Low,0.317980,0.271129,0.140899,"MULTIPOLYGON (((279989.84 3833209.559, 279989....",0.850073,0.805538,0.633842
3,172,2720.0,2.340000e+05,172,1.0,10,101,12,thm2017,thm2017_172,...,2,1,Low,0.362372,0.296424,0.126700,"MULTIPOLYGON (((279679.84 3833339.559, 279679....",0.844189,0.776932,0.527653
4,184,1540.0,4.900000e+04,184,1.0,10,100,11,thm2017,thm2017_184,...,3,1,Low,0.567295,0.592624,0.610778,"MULTIPOLYGON (((280029.84 3833179.559, 280029....",0.979764,0.982868,0.984800
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1731,44218,2880.0,2.577000e+05,44218,1.0,10,100,11,thm2017,thm2017_44218,...,2,1,Low,0.109455,0.073427,0.030402,"POLYGON ((294409.84 3796499.559, 294409.84 379...",0.341878,0.289806,0.234228
1732,44238,6260.0,8.933000e+05,44238,1.0,10,100,0,thm2017,thm2017_44238,...,2,1,Low,0.176968,0.177423,0.186665,"POLYGON ((297999.84 3797049.559, 297999.84 379...",0.434942,0.435691,0.450972
1733,44256,1140.0,3.360000e+04,44256,1.0,10,100,11,thm2017,thm2017_44256,...,2,1,Low,0.289667,0.279276,0.260739,"MULTIPOLYGON (((298689.84 3796789.559, 298699....",0.600808,0.583999,0.553547
1734,44320,3020.0,1.722000e+05,44320,1.0,10,0,0,thm2017,thm2017_44320,...,3,1,Low,0.542320,0.526273,0.466265,"POLYGON ((289169.84 3796289.559, 289169.84 379...",0.987247,0.985820,0.978950


In [22]:
merged_gdf.to_file(os.path.join(ODir, 'M1P_design_storm.shp'))